# Introduction to NLP fundamental using tensorflow
Deriving information from text or speech

or sequence 2 sequence generation

In [1]:
# !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data,create_tensorboard_callback, plot_loss_curves, compare_historys

## Get Data
twitter disaster

In [2]:
# !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-03-22 20:06:00--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.193.128, 209.85.202.128, 172.253.116.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  1.10MB/s    in 0.5s    

2023-03-22 20:06:01 (1.10 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [3]:
# unzip_data("nlp_getting_started.zip")

# Visualize, Visualize, Visualize

can use panda (limited by ram)
using tf load_text
using python loader

In [6]:
import pandas as pd

train_df = pd.read_csv("data/nlp_getting_started/train.csv")
test_df = pd.read_csv("data/nlp_getting_started/test.csv")

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [19]:
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

## split into validation

In [116]:
from sklearn.model_selection import train_test_split

train_text, val_text, train_label, val_label = train_test_split(train_df_shuffled["text"].to_numpy(),
                                train_df_shuffled["target"].to_numpy(), test_size=.1, random_state=42)

In [117]:
len(train_text), len(train_label), len(val_text), len(val_label)

(6851, 6851, 762, 762)

In [118]:
train_text[:10], train_label[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Vectorizing words


In [30]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [33]:
round(sum([len(i.split()) for i in train_text])/len(train_text)) # avg length of text

15

In [184]:
max_vocab_length =50000
max_length = 15

In [185]:
textvectorization = TextVectorization(max_tokens=max_vocab_length # length of volcabulary(rest to <OOV>)
                                      , standardize='lower_and_strip_punctuation',
    split='whitespace',ngrams=None,# ngram size
                                      output_mode='int', # to numbers
                                      output_sequence_length=max_length,# pad to max length
                                      pad_to_max_tokens=True  
                                     )

In [186]:
textvectorization.adapt(train_text)

2023-03-25 08:59:04.411718: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [181]:
sample = "There's a flood in my street"
textvectorization([sample])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [43]:
import random

In [48]:
random_choice = random.choice(train_text)
print(random_choice)
print(textvectorization([random_choice]))

#NP Metallica - Damage Inc
tf.Tensor(
[[2836    1  233 5376    0    0    0    0    0    0    0    0    0    0
     0]], shape=(1, 15), dtype=int64)


In [52]:
our_vocabulary = textvectorization.get_vocabulary()
len(our_vocabulary), our_vocabulary[:5], our_vocabulary[-5:]

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

# Creating embedding layer


In [243]:
embedding = tf.keras.layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128, # set /8 when using gpu
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=max_length
)

embedding

In [66]:
random_choice = random.choice(train_text)
print(random_choice)
vector = textvectorization([random_choice])
print(vector)
sample_embed = embedding(vector)

Sharing to help our cousin's family http://t.co/LSJowGYvQh
tf.Tensor(
[[4647    5  148  103 5902  302    1    0    0    0    0    0    0    0
     0]], shape=(1, 15), dtype=int64)


In [69]:
sample_embed[0][0], sample_embed[0][0].shape, random_choice

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.03114157,  0.01967374, -0.04736068, -0.01796595, -0.04870046,
        -0.0001228 , -0.03656517,  0.02118117, -0.02806422,  0.02665761,
        -0.02381406, -0.02557567, -0.048534  ,  0.02123635,  0.04308787,
         0.015731  ,  0.02820578,  0.03733463,  0.01771598,  0.04169016,
         0.04608223,  0.00220302, -0.0491954 ,  0.02065032, -0.04925536,
        -0.02303215, -0.04448877,  0.01592794, -0.04267229, -0.00565269,
        -0.03344625,  0.03074105, -0.04407436,  0.00965048,  0.00337793,
        -0.02072367,  0.00022598, -0.00970417,  0.00411699,  0.01119586,
        -0.04921091,  0.02989355,  0.02834091,  0.04475928, -0.0474855 ,
        -0.04036152, -0.00068638, -0.03608254, -0.02498015, -0.03486028,
         0.02828873,  0.03030254,  0.04037792, -0.04536163,  0.01363734,
        -0.00410252, -0.0279858 ,  0.0090971 ,  0.02767198,  0.02126597,
        -0.04666856, -0.02895921, -0.01100662, -0.02405472,  0.0262517 ,
  

# Experiment list

- naive bayes with tf-idf
- dense model
- LSTM(RNN)
- GRU (RNN) (gated recurrent unit)
- Bidrectional-LSTM (RNN)
- 1d CNN
- TF pretrained model feature extractor
- TF pretrained model feature extractor (10% data)

In [75]:

model_0 = tf.keras.models.Sequential()
model_0.add(tf.keras.layers.Input(shape=(1,), dtype=tf.string, name="input_layer"))

model_0.add(textvectorization)

model_0.add(embedding)



In [76]:
random_choice = random.choice(train_text)
model_0.predict([random_choice])

1/1 [==============================] - 0s 78ms/step


2023-03-23 07:37:48.208123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


array([[[ 0.03648401, -0.01343995,  0.02726519, ..., -0.01601354,
          0.0075595 , -0.03912764],
        [-0.00736792, -0.04412271,  0.03118983, ..., -0.01487361,
         -0.03070074,  0.04722324],
        [-0.03944248,  0.03668461, -0.02891522, ...,  0.01199384,
          0.03239999, -0.03269011],
        ...,
        [ 0.02208832,  0.04889195,  0.0108219 , ...,  0.00643492,
          0.00560063, -0.02738208],
        [ 0.03689495, -0.04735472,  0.01407543, ..., -0.03450272,
          0.04619033, -0.02157602],
        [-0.02808598, -0.01386461,  0.00114881, ...,  0.02699247,
          0.04818046, -0.01146827]]], dtype=float32)

## Baseline model 0
using sklearn navie bayes with tfidf

    its common to use non-dl algorithm first as baseline and then use dl to improve on it.

In [80]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("clf", MultinomialNB())
])

model_0.fit(train_text, train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [82]:
baseline_score = model_0.score(val_text, val_label)

In [83]:
baseline_score

0.7926509186351706

In [85]:
baseline_preds = model_0.predict(val_text)
baseline_preds[:10], val_label[:10]

(array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0]), array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0]))

In [99]:
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support

def get_metrcis(baseline_preds,val_label):
    Acc = tf.metrics.Accuracy()
    Acc.update_state(baseline_preds,val_label)
    acc = Acc.result().numpy()
    p, r, f, _ = precision_recall_fscore_support(baseline_preds,val_label, average="weighted") 
    return acc, p, r, f

In [100]:
get_metrcis(baseline_preds,val_label) 

(0.79265094, 0.8336022277575122, 0.7926509186351706, 0.7990828614653861)

In [101]:
from helper_functions import calculate_results, create_tensorboard_callback

In [102]:
calculate_results(baseline_preds,val_label)

{'accuracy': 79.26509186351706,
 'precision': 0.8336022277575122,
 'recall': 0.7926509186351706,
 'f1': 0.7990828614653861}

## Model 1 - using dense layers

In [140]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = textvectorization(inputs) # turn the input text into numbers

x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [141]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_17 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [142]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
# Get a summary of the model
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_17 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [143]:
# Fit the model
model_1_history = model_1.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_07", 
                                                                     experiment_name="07_model_1")])

Saving TensorBoard log files to: tensorboard_hub_07/07_model_1/20230324-075500
Epoch 1/5


2023-03-24 07:55:00.528855: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.6014 - accuracy: 0.7046

2023-03-24 07:55:05.801574: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 6s 25ms/step - loss: 0.6014 - accuracy: 0.7046 - val_loss: 0.5450 - val_accuracy: 0.7507
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4552 - accuracy: 0.8301 - val_loss: 0.4868 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3891 - accuracy: 0.8524 - val_loss: 0.4726 - val_accuracy: 0.7835
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3500 - accuracy: 0.8643 - val_loss: 0.4619 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3203 - accuracy: 0.8793 - val_loss: 0.4588 - val_accuracy: 0.7861


In [145]:
pred_probs = model_1.predict(val_text)

24/24 [==============================] - 0s 3ms/step


In [147]:
calculate_results(tf.squeeze(tf.round(pred_probs)), val_label)

{'accuracy': 78.60892388451444,
 'precision': 0.8081602243497674,
 'recall': 0.7860892388451444,
 'f1': 0.7899560900047056}

In [203]:
embedding_weights = model_1.get_layer("embedding_2").get_weights()
embedding_weights

[array([[-0.703444  , -0.65944374, -2.8652966 , ..., -1.0783211 ,
          0.09009187,  1.8993644 ],
        [-0.3668171 , -2.7066085 , -3.4043317 , ..., -3.8402143 ,
          1.8158928 ,  3.8957393 ],
        [ 0.0728832 ,  0.68576545,  0.9892081 , ..., -1.5324095 ,
         -2.2225018 ,  0.6028229 ],
        ...,
        [ 0.04161023, -0.04872613, -0.02946119, ...,  0.01696889,
          0.00421565, -0.01182178],
        [ 0.4400814 ,  0.16629887,  0.5899824 , ...,  0.21868843,
         -0.1266775 , -0.5093425 ],
        [ 0.7097707 , -0.8439144 ,  0.95670563, ...,  0.5303669 ,
          0.5005894 , -0.825076  ]], dtype=float32)]

In [205]:
embedding_weights[0].shape

(10000, 128)

Now we have embedding, let visualize it

using http://projector.tensorflow.org


also read https://www.tensorflow.org/text/guide/word_embeddings

In [208]:
# from https://www.tensorflow.org/text/guide/word_embeddings
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(our_vocabulary):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embedding_weights[0][index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

## Pratice model 2 lstm rnn

In [156]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = textvectorization(inputs) # turn the input text into numbers

x = embedding(x) # create an embedding of the numerized numbers
x = layers.LSTM(8)(x)


outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_2 = tf.keras.Model(inputs, outputs, name="model_2_dense") # construct the model

In [157]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
# Get a summary of the model
model_2.summary()

Model: "model_2_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_4 (LSTM)               (None, 8)                 4384      
                                                                 
 dense_19 (Dense)            (None, 1)                 9         
                                                                 
Total params: 1,284,393
Trainable params: 1,284,393
Non-trainable params: 0
___________________________________________

In [158]:
# Fit the model
model_2_history = model_2.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_07", 
                                                                     experiment_name="07_model_2")])

Saving TensorBoard log files to: tensorboard_hub_07/07_model_2/20230325-083434
Epoch 1/5


2023-03-25 08:34:35.239857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 08:34:35.398664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 08:34:35.679375: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.4015 - accuracy: 0.8324

2023-03-25 08:34:43.339200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 08:34:43.407442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 10s 37ms/step - loss: 0.4015 - accuracy: 0.8324 - val_loss: 0.4625 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.2299 - accuracy: 0.9206 - val_loss: 0.4893 - val_accuracy: 0.7966
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.1759 - accuracy: 0.9415 - val_loss: 0.5416 - val_accuracy: 0.7979
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.1481 - accuracy: 0.9491 - val_loss: 0.5741 - val_accuracy: 0.7992
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.1268 - accuracy: 0.9581 - val_loss: 0.5879 - val_accuracy: 0.7953


In [159]:
# !tensorboard dev upload --logdir ./tensorboard_hub_07 --name "07"    --description "disater" --one_shot


New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/GGG8QjeGT6exY2pQ3fEcYw/

[2023-03-25T08:37:20] Started scanning logdir.
[2023-03-25T08:38:05] Total uploaded: 366 scalars, 0 tensors, 19 binary objects (13.3 MB)
[2023-03-25T08:38:05] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/GGG8QjeGT6exY2pQ3fEcYw/


### using glove
https://keras.io/examples/nlp/pretrained_word_embeddings/


In [160]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip

--2023-03-25 08:49:02--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-03-25 08:49:02--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-03-25 08:49:03--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [161]:
textvectorization.get_vocabulary()[:5]

['', '[UNK]', 'the', 'a', 'in']

In [162]:
output = textvectorization([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([   2, 1427, 2721,   11,    2,    1])

In [187]:
voc = textvectorization.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [199]:
import os
import numpy as np
path_to_glove_file = os.path.join(
    os.path.expanduser("~"), "Personal Projects/code/MachineLearning/Udemy ztmdeeplearning/data/glove.6B/glove.6B.100d.txt"
)

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [201]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 11713 words (9343 misses)


In [202]:
del embeddings_index

In [191]:
from tensorflow.keras.layers import Embedding

prebuilt_embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [195]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = textvectorization(inputs) # turn the input text into numbers

x = prebuilt_embedding_layer(x) # create an embedding of the numerized numbers
x = layers.LSTM(8)(x)


outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_2_1 = tf.keras.Model(inputs, outputs, name="model_2_1_lstm") # construct the model

In [196]:
# Compile model
model_2_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
# Get a summary of the model
model_2_1.summary()

Model: "model_2_1_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_4 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 100)           2105800   
                                                                 
 lstm_6 (LSTM)               (None, 8)                 3488      
                                                                 
 dense_21 (Dense)            (None, 1)                 9         
                                                                 
Total params: 2,109,297
Trainable params: 3,497
Non-trainable params: 2,105,800
______________________________________

In [197]:
# Fit the model
model_2_1_history = model_2_1.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=20,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_07", 
                                                                     experiment_name="07_model_2_1")])

Saving TensorBoard log files to: tensorboard_hub_07/07_model_2_1/20230325-090247
Epoch 1/20


2023-03-25 09:02:49.321837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 09:02:49.469906: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 09:02:49.889047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.5895 - accuracy: 0.6913

2023-03-25 09:02:55.916290: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 09:02:55.991499: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 9s 30ms/step - loss: 0.5895 - accuracy: 0.6913 - val_loss: 0.5159 - val_accuracy: 0.7493
Epoch 2/20
215/215 [==============================] - 4s 18ms/step - loss: 0.4638 - accuracy: 0.7993 - val_loss: 0.4991 - val_accuracy: 0.7703
Epoch 3/20
215/215 [==============================] - 4s 17ms/step - loss: 0.4409 - accuracy: 0.8063 - val_loss: 0.4817 - val_accuracy: 0.7756
Epoch 4/20
215/215 [==============================] - 4s 19ms/step - loss: 0.4255 - accuracy: 0.8137 - val_loss: 0.4804 - val_accuracy: 0.7756
Epoch 5/20
215/215 [==============================] - 4s 19ms/step - loss: 0.4139 - accuracy: 0.8219 - val_loss: 0.4756 - val_accuracy: 0.7808
Epoch 6/20
215/215 [==============================] - 4s 18ms/step - loss: 0.4091 - accuracy: 0.8240 - val_loss: 0.4723 - val_accuracy: 0.7835
Epoch 7/20
215/215 [==============================] - 4s 18ms/step - loss: 0.4006 - accuracy: 0.8329 - val_loss: 0.4749 - val_accuracy: 0.7822
Epoch 8/20

## Recurrent neuron network

its useful for sequence data


it uses feedback from previous into 


https://deeplearning.mit.edu

http://introtodeeplearning.com

https://www.youtube.com/watch?v=ySEx_Bqxvvo&list=PLtBw6njQRU-rwp5__7C0oIVt26ZgjG9NI&index=2

https://colah.github.io/posts/2015-08-Understanding-LSTMs/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

## Model 2 - LSTM RNN

In [209]:
embedding = tf.keras.layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128, # set /8 when using gpu
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=max_length
)

embedding

In [210]:
textvectorization.adapt(train_text)

2023-03-25 11:11:43.681758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [220]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = textvectorization(inputs) # turn the input text into numbers

x = embedding(x) # create an embedding of the numerized numbers

# when stacking RNN's set return_sequences
# x = layers.LSTM(64, return_sequences=True)(x) # input - (None, 15, 128), output - (None, 15, 64) 

x = layers.LSTM(64)(x) # input - (None, 15, 64), output - (None, 15) 
# x = layers.Dense(64, activation="relu")(x)


outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_2 = tf.keras.Model(inputs, outputs, name="model_2_lstm") # construct the model

In [221]:
# Compile model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_2.summary()

Model: "model_2_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_13 (LSTM)              (None, 64)                49408     
                                                                 
 dense_28 (Dense)            (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [222]:
# Fit the model
model_2_history = model_2.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_2")])

Saving TensorBoard log files to: tensorboard_hub_08/08_model_2/20230325-112124
Epoch 1/5


2023-03-25 11:21:25.343083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:21:25.515752: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:21:25.948686: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.2366 - accuracy: 0.8980

2023-03-25 11:21:34.664773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:21:34.737850: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 11s 42ms/step - loss: 0.2366 - accuracy: 0.8980 - val_loss: 0.6348 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 5s 25ms/step - loss: 0.1131 - accuracy: 0.9588 - val_loss: 0.6755 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0905 - accuracy: 0.9680 - val_loss: 0.8712 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 5s 25ms/step - loss: 0.0786 - accuracy: 0.9721 - val_loss: 0.7429 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0695 - accuracy: 0.9739 - val_loss: 0.7880 - val_accuracy: 0.7730


In [223]:
model_2_pred_prods = model_2.predict(val_text)
model_2_pred_prods[:10]

2023-03-25 11:23:56.854196: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:23:56.911084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 1s 13ms/step


array([[1.6139824e-02],
       [9.5087600e-01],
       [9.9917400e-01],
       [8.1583951e-03],
       [4.6778444e-04],
       [9.9761760e-01],
       [9.9564362e-01],
       [9.9967742e-01],
       [9.9940693e-01],
       [4.5312569e-01]], dtype=float32)

## Model 3 - GRU RNN

similar to LSTM, but has less parameter !?


In [225]:
embedding = tf.keras.layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128, # set /8 when using gpu
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=max_length
)

embedding

In [244]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = textvectorization(inputs) # turn the input text into numbers

x = embedding(x) # create an embedding of the numerized numbers

# x = layers.GRU(64, return_sequences=True)(x)
# x = layers.LSTM(64, return_sequences=True)(x)
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x)

outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_3 = tf.keras.Model(inputs, outputs, name="model_3_gru") # construct the model

In [245]:
# Compile model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_3.summary()

Model: "model_3_gru"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_25 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           6400000   
                                                                 
 gru_8 (GRU)                 (None, 64)                37248     
                                                                 
 dense_33 (Dense)            (None, 1)                 65        
                                                                 
Total params: 6,437,313
Trainable params: 6,437,313
Non-trainable params: 0
_____________________________________________

In [242]:
# Fit the model
model_3_history = model_3.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_3")])

Saving TensorBoard log files to: tensorboard_hub_08/08_model_3/20230325-113440
Epoch 1/5


2023-03-25 11:34:41.327095: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:34:41.503382: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:34:41.934499: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.0896 - accuracy: 0.9726

2023-03-25 11:34:51.791846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:34:51.869355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 12s 48ms/step - loss: 0.0896 - accuracy: 0.9726 - val_loss: 0.9034 - val_accuracy: 0.7454
Epoch 2/5
215/215 [==============================] - 7s 32ms/step - loss: 0.0237 - accuracy: 0.9911 - val_loss: 0.8766 - val_accuracy: 0.7507
Epoch 3/5
215/215 [==============================] - 6s 30ms/step - loss: 0.0193 - accuracy: 0.9923 - val_loss: 0.8866 - val_accuracy: 0.7454
Epoch 4/5
215/215 [==============================] - 6s 29ms/step - loss: 0.0185 - accuracy: 0.9917 - val_loss: 1.2398 - val_accuracy: 0.7559
Epoch 5/5
215/215 [==============================] - 6s 29ms/step - loss: 0.0201 - accuracy: 0.9918 - val_loss: 1.0237 - val_accuracy: 0.7677


## Model 4 - Bidirectional RNN

In [246]:
embedding = tf.keras.layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128, # set /8 when using gpu
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=max_length
)

embedding

In [268]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = textvectorization(inputs) # turn the input text into numbers

x = embedding(x) # create an embedding of the numerized numbers

forward_layer = layers.LSTM(64
                           # , return_sequences=True
                           )
# backward_layer = layers.LSTM(64, activation='relu', return_sequences=True,
#                       go_backwards=True)


x = layers.Bidirectional(forward_layer)(x)

# x = layers.Dense(15)(x)

outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bi") # construct the model

In [270]:
# Compile model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_4.summary()

Model: "model_4_bi"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_40 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_6 (Embedding)     (None, 15, 128)           6400000   
                                                                 
 bidirectional_11 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dense_46 (Dense)            (None, 1)                 129       
                                                                 
Total params: 6,498,945
Trainable params: 6,498,945
Non-

In [271]:
# Fit the model
model_4_history = model_4.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_4")])

Saving TensorBoard log files to: tensorboard_hub_08/08_model_4/20230325-115252
Epoch 1/5


2023-03-25 11:52:54.196087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:52:54.473250: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:52:54.489773: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:52:55.030953: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:52:55.052171: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.5081 - accuracy: 0.7475

2023-03-25 11:53:09.246613: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:53:09.356803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-25 11:53:09.368298: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 18s 68ms/step - loss: 0.5081 - accuracy: 0.7475 - val_loss: 0.4657 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 9s 44ms/step - loss: 0.2464 - accuracy: 0.9024 - val_loss: 0.5276 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 9s 42ms/step - loss: 0.1126 - accuracy: 0.9603 - val_loss: 0.6831 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 9s 40ms/step - loss: 0.0592 - accuracy: 0.9834 - val_loss: 0.9481 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0434 - accuracy: 0.9874 - val_loss: 0.7594 - val_accuracy: 0.7690


## Model 5 - 1D CNN

In [272]:
embedding = tf.keras.layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128, # set /8 when using gpu
    embeddings_initializer='uniform',
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=max_length
)

embedding

In [342]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = textvectorization(inputs) # turn the input text into numbers

x = embedding(x) # create an embedding of the numerized numbers

x =  layers.Conv1D(64,5, activation='relu' )(x)

x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_5 = tf.keras.Model(inputs, outputs, name="model_5_conv1d") # construct the model

In [343]:
# Compile model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5.summary()

Model: "model_5_conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_75 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_7 (Embedding)     (None, 15, 128)           6400000   
                                                                 
 conv1d_32 (Conv1D)          (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_8 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_75 (Dense)            (None, 1)              

In [344]:
# Fit the model
model_5_history = model_5.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_5")])

Saving TensorBoard log files to: tensorboard_hub_08/08_model_5/20230326-074229
Epoch 1/5


2023-03-26 07:42:31.604407: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.0547 - accuracy: 0.9869

2023-03-26 07:42:42.219533: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 12s 49ms/step - loss: 0.0547 - accuracy: 0.9869 - val_loss: 0.8187 - val_accuracy: 0.7690
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0175 - accuracy: 0.9924 - val_loss: 0.8721 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0163 - accuracy: 0.9931 - val_loss: 0.8820 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0163 - accuracy: 0.9918 - val_loss: 0.9268 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0153 - accuracy: 0.9920 - val_loss: 0.9312 - val_accuracy: 0.7769


## Model 6 - pretrained embedding glove 1D CNN

In [346]:
# using prebuilt_embedding_layer glove

In [347]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = textvectorization(inputs) # turn the input text into numbers

x = prebuilt_embedding_layer(x) # create an embedding of the numerized numbers

x =  layers.Conv1D(64,5, activation='relu' )(x)

x = layers.GlobalMaxPool1D()(x)

outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_6_1 = tf.keras.Model(inputs, outputs, name="model_6_1_conv1d") # construct the model

In [348]:
# Compile model
model_6_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_6_1.summary()

Model: "model_5_conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_76 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_5 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_3 (Embedding)     (None, 15, 100)           2105800   
                                                                 
 conv1d_33 (Conv1D)          (None, 11, 64)            32064     
                                                                 
 global_max_pooling1d_9 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_76 (Dense)            (None, 1)              

In [349]:
# Fit the model
model_6_1_history = model_6_1.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_6_1")])

Saving TensorBoard log files to: tensorboard_hub_08/08_model_6_1/20230326-074652
Epoch 1/5


2023-03-26 07:46:53.919750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.4843 - accuracy: 0.7761

2023-03-26 07:46:58.352152: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 6s 20ms/step - loss: 0.4843 - accuracy: 0.7761 - val_loss: 0.4702 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3927 - accuracy: 0.8339 - val_loss: 0.4763 - val_accuracy: 0.7913
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.3435 - accuracy: 0.8584 - val_loss: 0.4579 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2979 - accuracy: 0.8847 - val_loss: 0.4681 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2610 - accuracy: 0.9044 - val_loss: 0.5132 - val_accuracy: 0.7874


## Model 6 - pretrained embedding USE 1D CNN

In [351]:
import tensorflow_hub as hub

USE_embedding_layer = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = USE_embedding_layer([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])

2023-03-26 07:58:13.632648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-03-26 07:58:14.530378: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [353]:
embeddings

<tf.Tensor: shape=(2, 512), dtype=float32, numpy=
array([[-0.03133017, -0.06338633, -0.01607502, ..., -0.0324278 ,
        -0.04575741,  0.05370455],
       [ 0.05080861, -0.01652432,  0.01573779, ...,  0.00976656,
         0.0317012 ,  0.01788118]], dtype=float32)>

In [356]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", input_shape=[],
                                       output_shape=[512], dtype="string", trainable=False, name="USE")

2023-03-26 08:08:03.041758: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [371]:
# Build model with the Functional API
from tensorflow.keras import layers
from tensorflow.keras import Sequential

model_6 = Sequential([
    sentence_encoder_layer,
    layers.Dense(54, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])



In [372]:
# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_6.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_82 (Dense)            (None, 54)                27702     
                                                                 
 dense_83 (Dense)            (None, 1)                 55        
                                                                 
Total params: 256,825,581
Trainable params: 27,757
Non-trainable params: 256,797,824
_________________________________________________________________


In [373]:
# Fit the model
model_6_history = model_6.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_6")])

Saving TensorBoard log files to: tensorboard_hub_08/08_model_6/20230326-081509
Epoch 1/5


2023-03-26 08:15:10.079629: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.5124 - accuracy: 0.7786

2023-03-26 08:15:39.936242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 38s 164ms/step - loss: 0.5124 - accuracy: 0.7786 - val_loss: 0.4483 - val_accuracy: 0.7966
Epoch 2/5
215/215 [==============================] - 20s 94ms/step - loss: 0.4163 - accuracy: 0.8127 - val_loss: 0.4368 - val_accuracy: 0.8163
Epoch 3/5
215/215 [==============================] - 19s 86ms/step - loss: 0.4023 - accuracy: 0.8216 - val_loss: 0.4341 - val_accuracy: 0.8136
Epoch 4/5
215/215 [==============================] - 19s 90ms/step - loss: 0.3938 - accuracy: 0.8270 - val_loss: 0.4316 - val_accuracy: 0.8123
Epoch 5/5
215/215 [==============================] - 19s 89ms/step - loss: 0.3875 - accuracy: 0.8273 - val_loss: 0.4335 - val_accuracy: 0.8150


In [369]:
baseline_score

0.7926509186351706

In [374]:
# Build model with the Functional API
from tensorflow.keras import layers
from tensorflow.keras import Sequential

model_6_2 = Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

# Compile model
model_6_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_6_2.summary()

# Fit the model
model_6_2_history = model_6_2.fit(train_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_6_2")])

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_84 (Dense)            (None, 128)               65664     
                                                                 
 dense_85 (Dense)            (None, 1)                 129       
                                                                 
Total params: 256,863,617
Trainable params: 65,793
Non-trainable params: 256,797,824
_________________________________________________________________
Saving TensorBoard log files to: tensorboard_hub_08/08_model_6_2/20230326-081922
Epoch 1/5


2023-03-26 08:19:23.319123: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - ETA: 0s - loss: 0.4854 - accuracy: 0.7867

2023-03-26 08:20:07.280476: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


215/215 [==============================] - 55s 239ms/step - loss: 0.4854 - accuracy: 0.7867 - val_loss: 0.4478 - val_accuracy: 0.8018
Epoch 2/5
215/215 [==============================] - 29s 134ms/step - loss: 0.4100 - accuracy: 0.8194 - val_loss: 0.4392 - val_accuracy: 0.8097
Epoch 3/5
215/215 [==============================] - 25s 115ms/step - loss: 0.3942 - accuracy: 0.8264 - val_loss: 0.4305 - val_accuracy: 0.8163
Epoch 4/5
215/215 [==============================] - 23s 109ms/step - loss: 0.3845 - accuracy: 0.8314 - val_loss: 0.4254 - val_accuracy: 0.8189
Epoch 5/5
215/215 [==============================] - 22s 101ms/step - loss: 0.3750 - accuracy: 0.8342 - val_loss: 0.4226 - val_accuracy: 0.8215


In [393]:
model_6_2.save_weights("08_model_6_2")

## Model 7 - 10% using USE

In [375]:
from sklearn.model_selection import train_test_split

train_90_text, train_10_text, train_90_label, train_10_label = train_test_split(train_text,
                                train_label, test_size=.1111, random_state=42)

In [376]:
# Build model with the Functional API
from tensorflow.keras import layers
from tensorflow.keras import Sequential

model_7 = Sequential([
    sentence_encoder_layer,
    layers.Dense(128, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_7.summary()

# Fit the model
model_7_history = model_7.fit(train_10_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_10_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_7")])

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_86 (Dense)            (None, 128)               65664     
                                                                 
 dense_87 (Dense)            (None, 1)                 129       
                                                                 
Total params: 256,863,617
Trainable params: 65,793
Non-trainable params: 256,797,824
_________________________________________________________________
Saving TensorBoard log files to: tensorboard_hub_08/08_model_7/20230326-082448
Epoch 1/5


2023-03-26 08:24:48.516673: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - ETA: 0s - loss: 0.6512 - accuracy: 0.6850

2023-03-26 08:25:02.381726: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 26s 996ms/step - loss: 0.6512 - accuracy: 0.6850 - val_loss: 0.6211 - val_accuracy: 0.7205
Epoch 2/5
24/24 [==============================] - 11s 471ms/step - loss: 0.5433 - accuracy: 0.8097 - val_loss: 0.5391 - val_accuracy: 0.7690
Epoch 3/5
24/24 [==============================] - 9s 382ms/step - loss: 0.4525 - accuracy: 0.8294 - val_loss: 0.4938 - val_accuracy: 0.7769
Epoch 4/5
24/24 [==============================] - 7s 313ms/step - loss: 0.3962 - accuracy: 0.8294 - val_loss: 0.4798 - val_accuracy: 0.7795
Epoch 5/5
24/24 [==============================] - 7s 293ms/step - loss: 0.3633 - accuracy: 0.8373 - val_loss: 0.4775 - val_accuracy: 0.7808


In [377]:
# Build model with the Functional API
from tensorflow.keras import layers
from tensorflow.keras import Sequential

model_7_1 = Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])

# Compile model
model_7_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_7_1.summary()

# Fit the model
model_7_1_history = model_7_1.fit(train_10_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_10_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_7_1")])

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_88 (Dense)            (None, 64)                32832     
                                                                 
 dense_89 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________
Saving TensorBoard log files to: tensorboard_hub_08/08_model_7_1/20230326-082627
Epoch 1/5


2023-03-26 08:26:28.148462: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - ETA: 0s - loss: 0.6622 - accuracy: 0.6929

2023-03-26 08:26:44.652723: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 29s 1s/step - loss: 0.6622 - accuracy: 0.6929 - val_loss: 0.6338 - val_accuracy: 0.7336
Epoch 2/5
24/24 [==============================] - 11s 466ms/step - loss: 0.5727 - accuracy: 0.8150 - val_loss: 0.5651 - val_accuracy: 0.7638
Epoch 3/5
24/24 [==============================] - 9s 390ms/step - loss: 0.4888 - accuracy: 0.8163 - val_loss: 0.5159 - val_accuracy: 0.7703
Epoch 4/5
24/24 [==============================] - 8s 354ms/step - loss: 0.4331 - accuracy: 0.8268 - val_loss: 0.4892 - val_accuracy: 0.7874
Epoch 5/5
24/24 [==============================] - 7s 313ms/step - loss: 0.3969 - accuracy: 0.8320 - val_loss: 0.4797 - val_accuracy: 0.7808


### This is wrong 
 spliting from original, creates dataset with val text in them ,  hence in calidating val accuracy

In [383]:
# Using original dataset
train_10_text = train_df_shuffled["text"].sample(frac=.1, random_state=42)
train_10_label = train_df_shuffled["target"].sample(frac=.1, random_state=42)

In [385]:
model_7_2 = tf.keras.models.clone_model(model_7_1)

model_7_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_7_2.summary()

# Fit the model
model_7_2_history = model_7_2.fit(train_10_text, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_10_label,
                              epochs=5,
                              validation_data=(val_text, val_label),
                              callbacks=[create_tensorboard_callback(dir_name="tensorboard_hub_08", 
                                                                     experiment_name="08_model_7_1")])

2023-03-26 08:35:02.000895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_88 (Dense)            (None, 64)                32832     
                                                                 
 dense_89 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________
Saving TensorBoard log files to: tensorboard_hub_08/08_model_7_1/20230326-083503
Epoch 1/5


2023-03-26 08:35:04.428669: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - ETA: 0s - loss: 0.6690 - accuracy: 0.6491

2023-03-26 08:35:20.522288: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 31s 1s/step - loss: 0.6690 - accuracy: 0.6491 - val_loss: 0.6326 - val_accuracy: 0.8045
Epoch 2/5
24/24 [==============================] - 12s 520ms/step - loss: 0.6007 - accuracy: 0.7976 - val_loss: 0.5562 - val_accuracy: 0.8045
Epoch 3/5
24/24 [==============================] - 10s 413ms/step - loss: 0.5265 - accuracy: 0.8003 - val_loss: 0.4890 - val_accuracy: 0.8031
Epoch 4/5
24/24 [==============================] - 9s 385ms/step - loss: 0.4713 - accuracy: 0.8042 - val_loss: 0.4421 - val_accuracy: 0.8163
Epoch 5/5
24/24 [==============================] - 8s 352ms/step - loss: 0.4326 - accuracy: 0.8147 - val_loss: 0.4120 - val_accuracy: 0.8281


In [386]:
model7_preds = model_7_2.predict(val_text)
model7_preds[:10]

2023-03-26 08:36:44.508638: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


24/24 [==============================] - 13s 495ms/step


array([[0.25420064],
       [0.7809625 ],
       [0.890362  ],
       [0.31554118],
       [0.7827334 ],
       [0.8166281 ],
       [0.868586  ],
       [0.9107538 ],
       [0.8135094 ],
       [0.05851967]], dtype=float32)

In [387]:
model7_preds = tf.squeeze(tf.round(model7_preds))
model7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [388]:
model7_results = calculate_results(val_label, model7_preds)

In [389]:
model7_results

{'accuracy': 82.80839895013123,
 'precision': 0.8281587277144384,
 'recall': 0.8280839895013123,
 'f1': 0.8275997984351715}

# Upload to board

In [394]:
# !tensorboard dev upload --logdir ./tensorboard_hub_08 --name "08"    --description "disater" --one_shot

# Finding most wrong examples

It hard to find most wrong in text data.

we use active learning to use trained model to get wrong examples.

In [396]:
model_6_2.evaluate(val_text, val_label)

24/24 [==============================] - 5s 171ms/step - loss: 0.4226 - accuracy: 0.8215


[0.4225907623767853, 0.8215222954750061]

In [399]:
pred_probs = model_6_2.predict(val_text)
val_pred = tf.squeeze(tf.round(pred_probs))

24/24 [==============================] - 5s 175ms/step


In [402]:
val_df = pd.DataFrame({
    "text": val_text,
    "target": val_label,
    "pred": val_pred,
    "pred_probs": tf.squeeze(pred_probs)
})

In [403]:
val_df.head()

,text,target,pred,pred_probs
0,DFR EP016 Monthly Meltdown - On Dnbheaven 2015...,0,0.0,0.238345
1,FedEx no longer to transport bioterror germs i...,0,1.0,0.759551
2,Gunmen kill four in El Salvador bus attack: Su...,1,1.0,0.992284
3,@camilacabello97 Internally and externally scr...,1,0.0,0.229405
4,Radiation emergency #preparedness starts with ...,1,1.0,0.743955


In [409]:
most_wrong = val_df[val_df["target"] != val_df["pred"]].sort_values(by=["pred_probs"], ascending=False)

In [410]:
most_wrong.head()

,text,target,pred,pred_probs
31,? High Skies - Burning Buildings ? http://t.co...,0,1.0,0.893212
759,FedEx will no longer transport bioterror patho...,0,1.0,0.876747
628,@noah_anyname That's where the concentration c...,0,1.0,0.871282
109,[55436] 1950 LIONEL TRAINS SMOKE LOCOMOTIVES W...,0,1.0,0.867149
49,@madonnamking RSPCA site multiple 7 story high...,0,1.0,0.838776


In [411]:
most_wrong.tail()

,text,target,pred,pred_probs
233,I get to smoke my shit in peace,1,0.0,0.049293
59,You can never escape me. Bullets don't harm me...,1,0.0,0.043141
244,Reddit Will Now QuarantineÛ_ http://t.co/pkUA...,1,0.0,0.042913
38,Why are you deluged with low self-image? Take ...,1,0.0,0.032078
23,Ron &amp; Fez - Dave's High School Crush https...,1,0.0,0.031326


# Speed/score tradeoff

In [416]:
import time

def pred_timer(model, samples):
    start_time = time.perf_counter()
    model.predict(samples)
    return time.perf_counter() - start_time    
    

In [417]:
pred_timer(model_6_2, val_text)

24/24 [==============================] - 4s 159ms/step


4.401971125000273

In [418]:
pred_timer(model_0, val_text)

0.03021912500844337

2023-03-26 10:20:11.885323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
